### Import Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical

### Data Loading

***Loading test-set***

In [ ]:
base_dir_test="input/Test"

In [ ]:
class_labels=os.listdir(base_dir_test)
print(class_labels)

In [ ]:
test_data=[]
count=0

for label in class_labels:
    i=0
    path = os.path.join(base_dir_test, label)
    print(path)
    for img in os.listdir(path):
        try:
            image=load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(256,256))
            image=img_to_array(image)
            image=image/255.0
            test_data.append([image,count])
        except Exception as e:
                pass
    count=count+1

In [ ]:
print(len(test_data))

In [ ]:
x_test,Y_test =zip(*test_data)

In [ ]:
x_test

In [ ]:
print(Y_test)

In [ ]:
pie_data=[]
for i in range(len(class_labels)):
    pie_data.append(list(Y_test).count(i))
with plt.style.context("ggplot"):
    plt.figure(figsize=(10,10))
    my_circle=plt.Circle( (0,0), 0.7, color='white')
    plt.pie(pie_data, labels=class_labels,autopct='%.2f%%')
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.show()

In [ ]:
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(18,8))
    plt.barh(width=pie_data,y=class_labels)
    plt.title(label="Records size comparision for classes")
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
X_test=np.array(x_test)
y_test=np.array(Y_test)

In [ ]:
print(X_test.shape,y_test.shape)

In [ ]:
from random import randint
def show_images(X,y):
    labels =class_labels
    x,y = X,y
    plt.figure(figsize=(15, 15))
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        idx = randint(0, X_test.shape[0]-1)
        plt.imshow(x[idx])
        plt.axis("off")
        plt.title("Class:{}".format(labels[y[idx]]))
show_images(X_test,y_test)

***Loading train-set***

In [ ]:
base_dir_train="input/Train"

In [ ]:
class_labels=os.listdir(base_dir_train)
print(class_labels)

In [ ]:
train_data=[]
count=0

for label in class_labels:
    i=0
    path = os.path.join(base_dir_train, label)
    print(path)
    for img in os.listdir(path):
        try:
            image=load_img(os.path.join(path, img), grayscale=False, color_mode='rgb', target_size=(256,256))
            image=img_to_array(image)
            image=image/255.0
            train_data.append([image,count])
        except Exception as e:
                pass
    count=count+1

In [ ]:
print(len(train_data))

In [ ]:
x_train,Y_train =zip(*train_data)

In [ ]:
x_train

In [ ]:
print(Y_train)

In [ ]:
X_train=np.array(x_train)
y_train=np.array(Y_train)

In [ ]:
print(X_train.shape,y_train.shape)

In [ ]:
from random import randint
def show_images(X,y):
    labels =class_labels
    x,y = X,y
    plt.figure(figsize=(15, 15))
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        idx = randint(0, X_train.shape[0]-1)
        plt.imshow(x[idx])
        plt.axis("off")
        plt.title("Class:{}".format(labels[y[idx]]))
show_images(X_train,y_train)

### Algorithm - ResNet152V2

In [ ]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizerslegacy import Adam

In [ ]:
resnet_model=tf.keras.applications.resnet_v2.ResNet152V2(include_top=False,
                                                   weights="imagenet",
                                                   input_tensor=None,
                                                   input_shape=(256,256,3),
                                                   pooling="max",
                                                   classes=1000)
resnet_model.trainable = False

In [ ]:
inputs = resnet_model.input
m = tf.keras.layers.Dense(512, activation='relu')(resnet_model.output)
outputs = tf.keras.layers.Dense(4, activation='softmax')(m)
resnet_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=1e-3,decay=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
history = resnet_model.fit(X_train, y_train, batch_size=64,validation_data = (X_test, y_test),epochs=10)

### Accuracy and Loss plot-graphs

In [ ]:
with plt.style.context(style='ggplot'):
    plt.figure(figsize=(18,8))
    plt.rcParams["font.size"]=15
    plt.plot(history.history["accuracy"],label="accuracy")
    plt.plot(history.history["val_accuracy"],label="val_accuracy")
    plt.title(label="Accuracy and Val_accuracy plot-graphs")
    plt.legend()
    plt.show()

    plt.figure(figsize=(18,8))
    plt.plot(history.history["loss"],label="loss")
    plt.plot(history.history["val_loss"],label="val_loss")
    plt.title(label="Loss and Val_loss plot-graphs")
    plt.legend()
    plt.show()

In [ ]:
resnet_model_prediction=resnet_model.predict(X_test,batch_size=64,verbose=1)

In [ ]:
print(resnet_model_prediction)

In [ ]:
resnet_model_labels=[]
for i in range(len(resnet_model_prediction)):
    resnet_model_labels.append(np.argmax(resnet_model_prediction[i]))

In [ ]:
print(resnet_model_labels)

In [ ]:
true_labels=[]
for i in range(len(y_test)):
    true_labels.append(np.argmax(y_test[i]))

In [ ]:
print(true_labels)

### Result Analysis

***Accuracy Score***

In [ ]:
resnet_accuracy=accuracy_score(y_true=true_labels,y_pred=resnet_model_labels)
print("ResNet152V2 model's validated accuracy is {:.2f}%".format(resnet_accuracy*100.0))

***Classification Report***

In [ ]:
print(classification_report(y_true=true_labels,y_pred=resnet_model_labels,target_names=class_labels))

***Confusion Matrix***

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
ax=plot_confusion_matrix(conf_mat=confusion_matrix(y_true=true_labels,y_pred=resnet_model_labels),
                        figsize=(10,5),
                        class_names=class_labels,
                        cmap=plt.cm.Reds)
plt.title(label="ResNet152V2 Confusion Matrix")
plt.xticks(rotation=90)
plt.show()

***Model Saving***

In [ ]:
resnet_model.save(filepath="models/ResNet152V2_model.h5")